# QEpy Fukui Si phases

## Nov.  06, 2023

pip install dftpy mpi4py mpi4py-fft qepy

In [1]:
from qepy.driver import Driver
from qepy.io import QEInput
from ase import Atoms
from ase.build import bulk
from ase.io import read

### Import needed DFTpy modules

In [2]:
import numpy as np

from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.functional import Functional, TotalFunctional, FunctionalOutput
from dftpy.functional.abstract_functional import AbstractFunctional
from dftpy.ions import Ions
from dftpy.constants import Units
from dftpy.formats import io
from dftpy.mpi import mp
from ase.io import read

## MPI setup

In [3]:
try :
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    mp.comm = MPI.COMM_WORLD
except:
    ## Serial version also can be done without mpi4py
    comm = None

In [4]:
dictionary = {
    'Si_Btin': {'file': 'Si_Btin.vasp'}, 
    'Si_fcc': {'file': 'Si_fcc.vasp'},
    'Si_bcc': {'file': 'Si_bcc.vasp'},
    'Si_cd': {'file': 'Si_cd.vasp'},
    'Si_dhcp': {'file': 'Si_dhcp.vasp'},
    'Si_bct5': {'file': 'Si_bct5.vasp'},
    'Si_sh': {'file': 'Si_sh.vasp'},
    'Si_cbcc': {'file': 'Si_cbcc.vasp'},
    'Si_hcp': {'file': 'Si_hcp.vasp'},
}

In [5]:
import os
os.chdir('../..')
path_file = os.getcwd()

In [6]:
def get_ions(phase):
    inputfile = path_file+'/Results/Structures/'+dictionary[phase]['file']
    ions = read(inputfile, format='vasp')
    return ions

In [7]:
# Phases = ['Si_Btin', 'Si_fcc', 'Si_bcc', 'Si_cd']#,'Si_dhcp' , 'Si_bct5', 'Si_cbcc', 'Si_sh']#, 'Si_hcp'] #]#
Phases = ['Si_Btin', 'Si_fcc', 'Si_bcc', 'Si_cd', 'Si_dhcp', 'Si_hcp', 'Si_bct5', 'Si_sh'] 
atoms = []
for i, phase in enumerate(Phases):
    atoms.append(get_ions(phase))

In [8]:
QE_options = []
for i, phase in enumerate(Phases):
    qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'/Users/valeria/Documents/PP/ofpp/EAC/upf/blps/'",
        'outdir': "'./tmp'",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc' : 60,
        'occupations': "'smearing'"
    },
    '&electrons': {
        'conv_thr' : 1e-8,
    },
    'atomic_species': ['Si  28.08 si.lda.upf'],
#     'k_points automatic': ['11 11 11 0 0 0'],
    'k_points automatic': ['8 8 8 0 0 0'],
}
    qe_options = QEInput().update_atoms(atoms[i], qe_options)
    QE_options.append(qe_options)

In [9]:
hartree = Functional(type='HARTREE')
Rho_opt_neutral = []
for i, phase in enumerate(np.arange(0,len(Phases))):
    print(i)
    qe_options = QE_options[i]
    driver=Driver(qe_options=qe_options, iterative = False, logfile='tmp.out')
    driver.scf()
    rho_f = driver.get_density()
    rho_opt = driver.data2field(rho_f) 
    driver.stop()
    Rho_opt_neutral.append(rho_opt)

    

0
1
2
3
4
5
6
7


### Run a full-SCF or directly read the converged density

In [10]:
# SCF
Rho_opt_charged = []
# ks_ke_opt = []
# ks_te_opt = []
for i, phase in enumerate(Phases):
    QE_options[i]['&system']['tot_charge']=0.1
    driver=Driver(qe_options=QE_options[i], iterative = False, logfile='tmp.out', comm=comm)
    driver.scf()
    rho_f = driver.get_density()
    rho_opt = driver.data2field(rho_f)
    ions = Ions.from_ase(atoms[i])
    Rho_opt_charged.append(rho_opt)

### Fukui

In [62]:
fg ={}
for i, phase in enumerate(Phases):
    fu = ((Rho_opt_charged[i]-Rho_opt_neutral[i])/0.1)
    fg[phase] = fu.fft()

In [23]:
M={}
Fukui = []
for i, phase in enumerate(Phases):
    fu = ((Rho_opt_charged[i]-Rho_opt_neutral[i])/0.1)-1
    fg = fu.fft()
    invg = fg.grid.gg
    mask = invg!=0
    invg[mask] = 1/invg[mask]
    fu_ha = ((fg.imag**2+fg.real**2)*4*np.pi*invg).integral()
    print(phase, fu_ha)
    M[phase] = fu_ha
    Fukui.append(np.abs(fu).integral())
    ions = Ions.from_ase(atoms[i])
    fu.write('/Users/valeria/Documents/aiWT/Final_version/wt/Results/Fukui/'+str(phase)+'_fukui.xsf', ions=ions)

Si_Btin 0.012869402427717834
Si_fcc 0.009901014540688438
Si_bcc 0.018353481705804104
Si_cd 0.015420205666489653
Si_dhcp 0.039983206542379725
Si_hcp 0.021900713969771408
Si_bct5 0.012912458532208479
Si_sh 0.06705876270601224


In [20]:
import pickle 

with open('/Users/valeria/Documents/aiWT/Final_version/wt/Results/Fukui/Fukui_hartree3.pkl', 'wb') as f:
    pickle.dump(M, f)

In [38]:
M={}
for i, phase in enumerate(Phases):
    fu = io.read_density('/Users/valeria/Documents/aiWT/Final_version/wt/Results/Fukui/'+str(phase)+'_fukui.xsf')
    fg = fu.fft()
    invg = fg.grid.gg
    mask = invg!=0
    invg[mask] = 1/invg[mask]
    fu_ha = ((fg.imag**2+fg.real**2)*4*np.pi*invg).integral()
    M[phase]=fu_ha
    Fukui.append(np.abs(fu).integral())